In [5]:
import requests
import mysql.connector

# Database configuration
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "tennis"
}

# API configuration
api_url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json"
api_key = "9GNLXLWScarfi2v3XZLs8QApmkYxPm9AfFVaVQNY"
headers = {
    "accept": "application/json"
}

# Connect to the database
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Fetch data from the API
params = {"api_key": api_key}
response = requests.get(api_url, headers=headers, params=params)
if response.status_code == 200:
    data = response.json()  # Assuming the response is in JSON format
    categories_inserted = set()  # To avoid duplicate category inserts

    for competitions in data['competitions']:
        # Insert into categories
        category = competitions['category']
        if category['id'] not in categories_inserted:
            cursor.execute("""
            INSERT IGNORE INTO categories (category_id, category_name) VALUES (%s, %s)
            """, (category['id'], category['name']))
            categories_inserted.add(category['id'])

        # Insert into competitions
        cursor.execute("""
        INSERT IGNORE INTO competitions (competition_id, competition_name, parent_id, type, gender, category_id)
        VALUES (%s, %s, %s, %s, %s, %s)
        """, (competitions['id'], competitions['name'], competitions.get('parent_id'), competitions['type'], competitions['gender'], category['id']))

    # Commit changes
    connection.commit()
    print("Data inserted successfully!")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}, Response: {response.text}")

# Close database connection
cursor.close()
connection.close()


Data inserted successfully!


In [ ]:
import requests
import mysql.connector

# Database configuration
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "tennis"
}

# API configuration
api_url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json"
api_key = "9GNLXLWScarfi2v3XZLs8QApmkYxPm9AfFVaVQNY"
headers = {
    "accept": "application/json"
}

try:
    # Connect to the database
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()

    # Fetch data from the API
    params = {"api_key": api_key}
    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()  # Assuming the response is in JSON format

        # A set to track inserted complex IDs to avoid duplicates
        complexes_inserted = set()

        for complex_item in data.get('complexes', []):  # Safely access 'complexes' key
            complex_id = complex_item.get('id')
            complex_name = complex_item.get('name')

            # Insert into complexes table
            if complex_id not in complexes_inserted:
                cursor.execute("""
                INSERT IGNORE INTO complexes (complex_id, complex_name)
                VALUES (%s, %s)
                """, (complex_id, complex_name))
                complexes_inserted.add(complex_id)

            # Process venues
            for venue in complex_item.get('venues', []):  # Iterate over the list of venues
                venue_id = venue.get('id')
                venue_name = venue.get('name')
                city_name = venue.get('city_name', '')  # Direct access to 'city_name'
                country_name = venue.get('country_name', '')  # Direct access to 'country_name'
                country_code = venue.get('country_code', '')  # Direct access to 'country_code'
                timezone = venue.get('timezone', '')  # Direct access to 'timezone'

                # Insert into venues table
                cursor.execute("""
                INSERT INTO venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                venue_name = VALUES(venue_name),
                city_name = VALUES(city_name),
                country_name = VALUES(country_name),
                country_code = VALUES(country_code),
                timezone = VALUES(timezone);
                """, (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id))
                

        # Commit changes to the database
        connection.commit()
        print("Data inserted successfully!")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Response: {response.text}")

except mysql.connector.Error as db_error:
    print(f"Database error: {db_error}")
except requests.RequestException as req_error:
    print(f"Request error: {req_error}")
finally:
    # Close the database connection
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()


In [ ]:
import requests
import mysql.connector

# Database configuration
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "tennis"
}

# API configuration
api_url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json"
api_key = "9GNLXLWScarfi2v3XZLs8QApmkYxPm9AfFVaVQNY"
headers = {"accept": "application/json"}

try:
    # Connect to the database
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()

    # Fetch data from the API
    params = {"api_key": api_key}
    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()

        # Process competitor_rankings only
        for ranking in data.get('rankings', []):  # Loop through rankings
            for competitor_ranking in ranking.get('competitor_rankings', []):
                # Extract competitor details
                competitor = competitor_ranking.get('competitor', {})
                competitor_id = competitor.get('id', '')
                competitor_name = competitor.get('name', '')
                country = competitor.get('country', '')
                country_code = competitor.get('country_code', '')
                abbreviation = competitor.get('abbreviation', '')

                # Insert competitor details into the competitors table
                cursor.execute("""
                    INSERT INTO competitors (
                        competitor_id, name, country, country_code, abbreviation
                    )
                    VALUES (%s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                        name = VALUES(name),
                        country = VALUES(country),
                        country_code = VALUES(country_code),
                        abbreviation = VALUES(abbreviation);
                """, (competitor_id, competitor_name, country, country_code, abbreviation))

                # Extract ranking details
                rank = competitor_ranking.get('rank', 0)
                movement = competitor_ranking.get('movement', 0)
                points = competitor_ranking.get('points', 0)
                competitions_played = competitor_ranking.get('competitions_played', 0)

                # Insert ranking details into the competitor_rankings table
                cursor.execute("""
                    INSERT INTO competitor_rankings (
                        rank, movement, points, competitions_played, competitor_id
                    )
                    VALUES (%s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                        rank = VALUES(rank),
                        movement = VALUES(movement),
                        points = VALUES(points),
                        competitions_played = VALUES(competitions_played);
                """, (rank, movement, points, competitions_played, competitor_id))

                # Debug log for each competitor processed
                print(f"Processed Competitor: {competitor_name}, Rank: {rank}, Points: {points}")

        # Commit changes to the database
        connection.commit()
        print("Competitor and ranking data inserted successfully!")

    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Response: {response.text}")

except mysql.connector.Error as db_error:
    print(f"Database error: {db_error}")
except requests.RequestException as req_error:
    print(f"Request error: {req_error}")
finally:
    # Close the database connection
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()
